In [6]:
# Data visualization
import numpy as np
import pandas as pd

# TensorFlow Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

## 1. Important Landmarks and Important functions

In [7]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE"
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [8]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

## 2. Describe Dataset & Split Data

2.1 Make Input Scaler

In [9]:
from sklearn.preprocessing import StandardScaler

# Load dataset
df = describe_dataset("./train.csv")

# Initialize the scaler
sc = StandardScaler()

# Fit the scaler on the data (excluding the label column)
sc.fit(df.drop("label", axis=1))

# Save the scaler to a file
with open("./model/input_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 4160 
Number of columns: 37

Labels: 
down    2127
up      2033
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


2.2 Describe Dataset & Split Data

In [10]:
# load dataset
df = describe_dataset("./train.csv")

# Categorizing label
df.loc[df["label"] == "down", "label"] = 0
df.loc[df["label"] == "up", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 4160 
Number of columns: 37

Labels: 
down    2127
up      2033
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [11]:
with open("./model/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [12]:
# Standard Scaling of features
x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

## 3. Build Model

### 3.1. Set up

In [14]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

# Final Results
final_models = {}

In [15]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

### 3.2. Model with 3 layers

In [16]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [17]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo',
)
tuner_3l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 02s]
val_accuracy: 0.9963942170143127

Best val_accuracy So Far: 0.9987980723381042
Total elapsed time: 00h 00m 34s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [18]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17917b9d0>
Layer-2: 224 units, func:  <function relu at 0x17917b9d0>
Layer-3: 2 units, func:  <function softmax at 0x179173f70>

Other params:
learning_rate: 0.001
Epoch 1/100
333/333 [==============================] - 1s 1ms/step - loss: 0.0287 - accuracy: 0.9943 - val_loss: 0.0118 - val_accuracy: 0.9964
Epoch 2/100
333/333 [==============================] - 0s 760us/step - loss: 0.0031 - accuracy: 0.9988 - val_loss: 0.0077 - val_accuracy: 0.9976
Epoch 3/100
333/333 [==============================] - 0s 727us/step - loss: 0.0017 - accuracy: 0.9994 - val_loss: 0.0096 - val_accuracy: 0.9964
Epoch 4/100
333/333 [==============================] - 0s 758us/step - loss: 7.5397e-04 - accuracy: 1.0000 - val_loss: 0.0076 - val_accuracy: 0.9976
Epoch 5/100
333/333 [==============================] - 0s 744us/step - loss: 3.0912e-04 - accuracy: 1.0000 - val_loss: 0.0124 - val_accuracy: 0.9964
Epoch 6/100
333/333 [=========

In [19]:
final_models["3_layers"] = model_3l

### 3.3. Model with 5 layers

In [20]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [21]:
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_2'
)
tuner_5l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 04s]
val_accuracy: 0.9987980723381042

Best val_accuracy So Far: 0.9987980723381042
Total elapsed time: 00h 01m 06s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [22]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17917b9d0>
Layer-2: 64 units, func:  <function relu at 0x17917b9d0>
Layer-3: 480 units, func:  <function relu at 0x17917b9d0>
Layer-4: 192 units, func:  <function relu at 0x17917b9d0>
Layer-5: 2 units, func:  <function softmax at 0x179173f70>

Other params:
learning_rate: 0.001
Epoch 1/100
333/333 [==============================] - 2s 2ms/step - loss: 0.0309 - accuracy: 0.9940 - val_loss: 0.0090 - val_accuracy: 0.9952
Epoch 2/100
333/333 [==============================] - 1s 2ms/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.0120 - val_accuracy: 0.9964
Epoch 3/100
333/333 [==============================] - 1s 2ms/step - loss: 0.0022 - accuracy: 0.9991 - val_loss: 0.0172 - val_accuracy: 0.9952
Epoch 4/100
333/333 [==============================] - 1s 2ms/step - loss: 2.8212e-04 - accuracy: 1.0000 - val_loss: 0.0422 - val_accuracy: 0.9952


In [23]:
final_models["5_layers"] = model_5l

### 3.4. Model with 7 layers include Dropout

In [24]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [25]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_3'
)
tuner_7lD.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 02s]
accuracy: 0.9987980723381042

Best accuracy So Far: 1.0
Total elapsed time: 00h 01m 07s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [26]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17917b9d0>
Layer-2: 192 units, func:  <function relu at 0x17917b9d0>
Layer-3: 0 units, func: None
Layer-4: 64 units, func:  <function relu at 0x17917b9d0>
Layer-5: 0 units, func: None
Layer-6: 64 units, func:  <function relu at 0x17917b9d0>
Layer-7: 2 units, func:  <function softmax at 0x179173f70>

Other params:
dropout_1: 0.1
dropout_2: 0.30000000000000004
learning_rate: 0.001
Epoch 1/100
333/333 [==============================] - 2s 2ms/step - loss: 0.0340 - accuracy: 0.9904 - val_loss: 0.0040 - val_accuracy: 0.9988
Epoch 2/100
333/333 [==============================] - 0s 947us/step - loss: 0.0051 - accuracy: 0.9979 - val_loss: 0.0054 - val_accuracy: 0.9988
Epoch 3/100
333/333 [==============================] - 0s 954us/step - loss: 0.0033 - accuracy: 0.9991 - val_loss: 0.0170 - val_accuracy: 0.9964
Epoch 4/100
333/333 [==============================] - 0s 944us/step - loss: 6.4516e-04 - accuracy: 1.0000 - v

In [27]:
final_models["7_layers_with_dropout"] = model_7lD

### 3.5. Model with 7 layers

In [28]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [29]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_6'
)
tuner_7l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 04s]
val_accuracy: 0.9975961446762085

Best val_accuracy So Far: 0.9987980723381042
Total elapsed time: 00h 01m 25s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [30]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17917b9d0>
Layer-2: 32 units, func:  <function relu at 0x17917b9d0>
Layer-3: 416 units, func:  <function relu at 0x17917b9d0>
Layer-4: 288 units, func:  <function relu at 0x17917b9d0>
Layer-5: 320 units, func:  <function relu at 0x17917b9d0>
Layer-6: 480 units, func:  <function relu at 0x17917b9d0>
Layer-7: 2 units, func:  <function softmax at 0x179173f70>

Other params:
learning_rate: 0.001
Epoch 1/100
333/333 [==============================] - 3s 4ms/step - loss: 0.0439 - accuracy: 0.9892 - val_loss: 0.0237 - val_accuracy: 0.9964
Epoch 2/100
333/333 [==============================] - 1s 4ms/step - loss: 0.0104 - accuracy: 0.9976 - val_loss: 0.0029 - val_accuracy: 0.9988
Epoch 3/100
333/333 [==============================] - 2s 5ms/step - loss: 0.0023 - accuracy: 0.9988 - val_loss: 0.0213 - val_accuracy: 0.9976
Epoch 4/100
333/333 [==============================] - 2s 5ms/step - loss: 0.0068 - accuracy: 0.9988 

In [31]:
final_models["7_layers"] = model_7l

### 3.6. Describe final models

In [32]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17917b9d0>
Layer-2: 224 units, func:  <function relu at 0x17917b9d0>
Layer-3: 2 units, func:  <function softmax at 0x179173f70>

5_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17917b9d0>
Layer-2: 64 units, func:  <function relu at 0x17917b9d0>
Layer-3: 480 units, func:  <function relu at 0x17917b9d0>
Layer-4: 192 units, func:  <function relu at 0x17917b9d0>
Layer-5: 2 units, func:  <function softmax at 0x179173f70>

7_layers_with_dropout: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17917b9d0>
Layer-2: 192 units, func:  <function relu at 0x17917b9d0>
Layer-3: 0 units, func: None
Layer-4: 64 units, func:  <function relu at 0x17917b9d0>
Layer-5: 0 units, func: None
Layer-6: 64 units, func:  <function relu at 0x17917b9d0>
Layer-7: 2 units, func:  <function softmax at 0x179173f70>

7_layers: Describe models architecture
Layer-1: 36 units, 

## 4. Model Evaluation

### 4.1. Train set evaluation

In [33]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers,"[0.998, 1.0]","[1.0, 0.998]","[0.999, 0.999]","[[427, 0], [1, 404]]"
1,7_layers_with_dropout,"[0.995, 1.0]","[1.0, 0.995]","[0.998, 0.998]","[[427, 0], [2, 403]]"
2,3_layers,"[0.993, 1.0]","[1.0, 0.993]","[0.996, 0.996]","[[427, 0], [3, 402]]"
3,5_layers,"[0.991, 1.0]","[1.0, 0.99]","[0.995, 0.995]","[[427, 0], [4, 401]]"


### 4.2. Test set evaluation

In [34]:
# load dataset
test_df = describe_dataset("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "down", "label"] = 0
test_df.loc[test_df["label"] == "up", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 853 
Number of columns: 37

Labels: 
down    430
up      423
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [35]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [36]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: k[1] + k[2] + k[3], reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout,"[0.995, 0.998]","[0.998, 0.995]","[0.997, 0.996]","[[429, 1], [2, 421]]"
1,7_layers,"[0.993, 0.998]","[0.998, 0.993]","[0.995, 0.995]","[[429, 1], [3, 420]]"
2,3_layers,"[0.993, 0.995]","[0.995, 0.993]","[0.994, 0.994]","[[428, 2], [3, 420]]"
3,5_layers,"[0.988, 0.998]","[0.998, 0.988]","[0.993, 0.993]","[[429, 1], [5, 418]]"


## 5. Dumped Model

In [39]:
# Dump the best model to a pickle file
with open("./model/squat_dp.pkl", "wb") as f:
    pickle.dump(final_models["7_layers_with_dropout"], f)

In [40]:
# Dump final results
with open("./model/all_models.pkl", "wb") as f:
    pickle.dump(final_models, f)